In [ ]:
import os
import numpy as np
import shutil
import json
import pandas as pd
import math
from datetime import *
from dateutil import parser
import itertools
from collections import defaultdict
import ast
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
dataset = 'brackets'

# Process raw data 
a JSON file

In [ ]:
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/'+dataset+'.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'subProject': '', 'changeId': '157038733', 'changeNumber': 13964, 'timestamp': 1512660020000, 'reviewers': [{'accountId': '1577184', 'email': '', 'name': 'nethip', 'avatar': 'https://avatars2.githubusercontent.com/u/1577184?v=4'}], 'owner': {'accountId': '7641760', 'email': '', 'name': 'petetnt', 'avatar': 'https://avatars2.githubusercontent.com/u/7641760?v=4'}, 'filePaths': [{'location': 'src/document/DocumentCommandHandlers.js'}]}
{'subProject': '', 'changeId': '155414893', 'changeNumber': 13946, 'timestamp': 1511975683000, 'reviewers': [{'accountId': '26472019', 'email': '', 'name': 'JeongGoEun', 'avatar': 'https://avatars2.githubusercontent.com/u/26472019?v=4'}], 'owner': {'accountId': '26391179', 'email': '', 'name': 'vutondesign', 'avatar': 'https://avatars3.githubusercontent.com/u/26391179?v=4'}, 'filePaths': [{'location': 'README.md'}]}
{'subProject': '', 'changeId': '155305168', 'changeNumber': 13945, 'timestamp': 1511948699000, 'reviewers': [{'accountId': '16770756', 'email'

In [ ]:
len(data)

4264

In [ ]:
# get an element of the list of data
data[0]

{'changeId': '157038733',
 'changeNumber': 13964,
 'filePaths': [{'location': 'src/document/DocumentCommandHandlers.js'}],
 'owner': {'accountId': '7641760',
  'avatar': 'https://avatars2.githubusercontent.com/u/7641760?v=4',
  'email': '',
  'name': 'petetnt'},
 'reviewers': [{'accountId': '1577184',
   'avatar': 'https://avatars2.githubusercontent.com/u/1577184?v=4',
   'email': '',
   'name': 'nethip'}],
 'subProject': '',
 'timestamp': 1512660020000}

In [ ]:
# read a single feature of an element
data[0]['reviewers']

[{'accountId': '1577184',
  'avatar': 'https://avatars2.githubusercontent.com/u/1577184?v=4',
  'email': '',
  'name': 'nethip'}]

In [ ]:
# Extract and save desired features from each pull request. The new desired data is new_data
new_data = []

for i in range(0, len(data)):
  elem_dict = {"changeId":data[i]['changeId'], "reviewers":data[i]['reviewers'], "owner":data[i]['owner'], "filePaths":data[i]['filePaths'], "time":data[i]['timestamp']}
  new_data.append(elem_dict)
  # print(new_data[i])

print(len(new_data))

4264


In [ ]:
new_data[0]

{'changeId': '157038733',
 'filePaths': [{'location': 'src/document/DocumentCommandHandlers.js'}],
 'owner': {'accountId': '7641760',
  'avatar': 'https://avatars2.githubusercontent.com/u/7641760?v=4',
  'email': '',
  'name': 'petetnt'},
 'reviewers': [{'accountId': '1577184',
   'avatar': 'https://avatars2.githubusercontent.com/u/1577184?v=4',
   'email': '',
   'name': 'nethip'}],
 'time': 1512660020000}

In [ ]:
# Remove remaining undesired features. The final desired information is saved in final_data
final_data = []

for i in range(0, len(new_data)):
  if len(new_data[i]['filePaths']) != 0:
    elem_dict = {"time":new_data[i]['time'], "changeId":new_data[i]['changeId'], "ownerId": new_data[i]['owner']['accountId'], "reviewersId":[], "filePaths":[]}

    for j in range(0, len(new_data[i]['reviewers'])):
      elem_dict["reviewersId"].append(new_data[i]['reviewers'][j]['accountId'])

    for j in range(0, len(new_data[i]['filePaths'])):
      elem_dict["filePaths"].append(new_data[i]['filePaths'][j]['location'])
    
    final_data.append(elem_dict)
    # print(final_data[i])

print(len(final_data))

4254


In [ ]:
final_data[0]

{'changeId': '157038733',
 'filePaths': ['src/document/DocumentCommandHandlers.js'],
 'ownerId': '7641760',
 'reviewersId': ['1577184'],
 'time': 1512660020000}

In [ ]:
# change timestamp value from int to date format
for pr in final_data:
  timestamp = int(pr['time']/1000)  # removing the 3 last zero digits 
  dt_obj = datetime.fromtimestamp(timestamp) 
  pr['time'] = dt_obj

final_data[0]

{'changeId': '157038733',
 'filePaths': ['src/document/DocumentCommandHandlers.js'],
 'ownerId': '7641760',
 'reviewersId': ['1577184'],
 'time': datetime.datetime(2017, 12, 7, 15, 20, 20)}

In [ ]:
# sort data by time
sorted_data = sorted(final_data, key=lambda d: d['time'])

# Get the Time of available data from selected project
print("Start date:", sorted_data[0]['time'])
print("End date:", sorted_data[-1]['time'])

Start date: 2011-12-14 22:54:49
End date: 2017-12-07 15:20:20


In [ ]:
sorted_data = [pr for pr in sorted_data if (pr['time'] <= datetime(2016, 7, 31, 23, 59, 59))]
len(sorted_data)

3967

In [ ]:
# Convert list to json
# It should be a string before write it to file
# "default" is added to dumps to make anything not serializable for JSON to string format (like datatime) 
json_string = json.dumps(sorted_data, default=str) 

# Save OpenStack new dataset
file = open(dataset+'_final.json', 'w')
file.write(json_string)
file.close()

In [ ]:
src = "./"+dataset+"_final.json"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/"+dataset+"_final.json"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/brackets_final.json'

# Implementation

## Load and Clean data

In [ ]:
# Load data from drive
f = open('/content/drive/MyDrive/Colab Notebooks/My Project/'+dataset+'_final.json',)
data = json.load(f)  # returns json object as a dictionary
for i in range(5):   # 5 first elements of the json list
    print(data[i])
f.close()

{'time': '2011-12-14 22:54:49', 'changeId': '596123', 'ownerId': '1184639', 'reviewersId': ['1060119'], 'filePaths': ['src/brackets.js', 'test/spec/NativeFileSystem-test.js', 'src/NativeFileSystem.js']}
{'time': '2011-12-15 22:37:59', 'changeId': '600880', 'ownerId': '697759', 'reviewersId': ['1060119'], 'filePaths': ['test/spec/LowLevelFileIO-test.js']}
{'time': '2011-12-16 23:14:14', 'changeId': '605484', 'ownerId': '1060119', 'reviewersId': ['1148713'], 'filePaths': ['test/spec/LowLevelFileIO-test.js']}
{'time': '2011-12-19 21:31:24', 'changeId': '612046', 'ownerId': '1197510', 'reviewersId': ['458642'], 'filePaths': ['.gitignore']}
{'time': '2011-12-19 23:46:20', 'changeId': '612574', 'ownerId': '1148713', 'reviewersId': ['1060119'], 'filePaths': ['test/spec/FileCommandHandlers-test-files/test.js', 'src/FileCommandHandlers.js', 'test/SpecRunner.html', 'test/spec/FileCommandHandlers-test.js', 'test/spec/SpecRunnerUtils.js']}


In [ ]:
# convert times to datetime format
for pr in data:
  pr['time'] = parser.parse(pr['time'])

data[0]

{'changeId': '596123',
 'filePaths': ['src/brackets.js',
  'test/spec/NativeFileSystem-test.js',
  'src/NativeFileSystem.js'],
 'ownerId': '1184639',
 'reviewersId': ['1060119'],
 'time': datetime.datetime(2011, 12, 14, 22, 54, 49)}

In [ ]:
len(data)

3967

In [ ]:
# Save only PRs with 1 reviwer (to make the results similar to the paper)
final_data = []
for pr in data:
  if len(pr['reviewersId']) > 1:
    revs = pr['reviewersId']
    for r in revs:
      pr['reviewersId'] = [r]
      final_data.append(pr)
  else:
    final_data.append(pr)

len(final_data)

6314

## Approach

In [ ]:
def get_all_reviewers():
  # create a set of all reviewers
  all_reviewers = set()   
  for pr in final_data:
    for r in pr['reviewersId']:
      all_reviewers.add(r)
  return all_reviewers

all_reviewers = list(get_all_reviewers())
print(all_reviewers)
print(len(all_reviewers))

['5047365', '1514099', '3188532', '1299997', '2292405', '3670146', '297661', '3427344', '2284377', '9378357', '55970', '1184639', '2338268', '1416', '86214', '554953', '378835', '5663877', '5307412', '4616705', '905006', '374857', '2781554', '2150814', '1197545', '14019856', '11662297', '9530214', '441774', '7641760', '546845', '2631719', '1197510', '5497320', '3382464', '5336369', '1437914', '6702631', '4721259', '10656375', '72777', '237899', '1774931', '4241017', '7239988', '2170694', '37918', '5073477', '1148713', '108065', '1067319', '1234944', '3157742', '1189142', '2547815', '384845', '1326923', '1378718', '287917', '141435', '1701706', '1457701', '1075154', '500893', '908679', '697759', '2119748', '2037113', '1072425', '10137', '306818', '864201', '367710', '1449826', '5213421', '12087205', '2069528', '1486879', '1584287', '1597992', '1495261', '1403587', '1049908', '307660', '7549136', '5946409', '3496166', '1041792', '70010', '10583268', '1577184', '3032628', '2816999', '2641

In [ ]:
all_reviewers = ['5047365', '1514099', '3188532', '1299997', '2292405', '3670146', '297661', '3427344', '2284377', '9378357', '55970', '1184639', '2338268', '1416', '86214', '554953', '378835', '5663877', '5307412', '4616705', '905006', '374857', '2781554', '2150814', '1197545', '14019856', '11662297', '9530214', '441774', '7641760', '546845', '2631719', '1197510', '5497320', '3382464', '5336369', '1437914', '6702631', '4721259', '10656375', '72777', '237899', '1774931', '4241017', '7239988', '2170694', '37918', '5073477', '1148713', '108065', '1067319', '1234944', '3157742', '1189142', '2547815', '384845', '1326923', '1378718', '287917', '141435', '1701706', '1457701', '1075154', '500893', '908679', '697759', '2119748', '2037113', '1072425', '10137', '306818', '864201', '367710', '1449826', '5213421', '12087205', '2069528', '1486879', '1584287', '1597992', '1495261', '1403587', '1049908', '307660', '7549136', '5946409', '3496166', '1041792', '70010', '10583268', '1577184', '3032628', '2816999', '2641501', '2761359', '1542488', '5414378', '4968007', '45450', '1060119', '78037', '171178', '1439163', '5271543', '296940', '887904', '619436', '425227', '177243', '827019', '1283787', '1795016', '1212885', '1564859', '976619', '4923837', '5496284', '10204469', '1131747', '8229975', '2753459', '125516', '3368515', '4304536', '197418', '1025666', '1061323', '6923008', '11760209', '7630875', '169885', '5547669', '1726914', '2771653', '539561', '4276074', '4999744', '694940', '2568208', '1197144', '547312', '27985', '458642', '3752015', '5816130']

print("All Reviewers in project:", all_reviewers)
print("Number of Reviewers in whole project:", len(all_reviewers))

All Reviewers in project: ['5047365', '1514099', '3188532', '1299997', '2292405', '3670146', '297661', '3427344', '2284377', '9378357', '55970', '1184639', '2338268', '1416', '86214', '554953', '378835', '5663877', '5307412', '4616705', '905006', '374857', '2781554', '2150814', '1197545', '14019856', '11662297', '9530214', '441774', '7641760', '546845', '2631719', '1197510', '5497320', '3382464', '5336369', '1437914', '6702631', '4721259', '10656375', '72777', '237899', '1774931', '4241017', '7239988', '2170694', '37918', '5073477', '1148713', '108065', '1067319', '1234944', '3157742', '1189142', '2547815', '384845', '1326923', '1378718', '287917', '141435', '1701706', '1457701', '1075154', '500893', '908679', '697759', '2119748', '2037113', '1072425', '10137', '306818', '864201', '367710', '1449826', '5213421', '12087205', '2069528', '1486879', '1584287', '1597992', '1495261', '1403587', '1049908', '307660', '7549136', '5946409', '3496166', '1041792', '70010', '10583268', '1577184', '

In [ ]:
# problem hyper-parameters
lambda_val = 7  # can be changed
gama = 0.7

### Time-decay Relationship

In [ ]:
def find_PRlist(pull_reqs, reviewerId, cur_time):

  # lambda days before the time of our test PR
  lambda_ago = cur_time - timedelta(days=lambda_val)

  # list of PRs in "pull_reqs" that are between time of lambda days ago up to the time of test PR
  PS = [pr for pr in pull_reqs if ((lambda_ago < pr['time'] < cur_time) and (reviewerId in pr['reviewersId']))]
  # print("size of final list:", len(PS))

  # PS will be the list of PRs which submitted by the owner of test PR and reviewed by any reviewer in lambda days ago
  return PS 

In [ ]:
def cal_decayRelation(pull_reqs, cur_time):
  return sum([(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

### Time-decay File Similarity

In [ ]:
def pathDistance(f1, f2):
  
  commen_prefix = os.path.commonprefix([f1, f2]).split("/")
  commen_len = len(commen_prefix)
  if '' in commen_prefix:
   commen_len -= 1
  # print(commen_prefix, commen_len)

  max_len = max(len(f1.split("/")), len(f2.split("/")))
  # print(max_len)

  return commen_len / max_len

In [ ]:
def prDistance(pr1, pr2):
  return sum([pathDistance(f1, f2) for f1 in pr1['filePaths'] for f2 in pr2['filePaths']]) / (len(pr1['filePaths'])*len(pr2['filePaths']))

In [ ]:
def cal_decayFileSimilarity(pull_reqs, cur_time, test_pr):
  return sum([prDistance(test_pr, pr)*(1/(cur_time-pr['time']).days) for pr in pull_reqs if ((cur_time-pr['time']).days!=0)])

### Feature Extraction
Construct a Pull Request Vector for each PR

In [ ]:
# For all PRs we should extract PRvector and save this and reviewer ID into a dataframe for future use

vectors = []
labels = []

for t in range(0, len(final_data)):

  prev_pulls = final_data[0:t]
  cur_pull = final_data[t]

  # list of PRs that submitted by the owner of this PR in trainset
  OS = [pr for pr in prev_pulls if (pr['ownerId'] == cur_pull['ownerId'])]

  cur_time = cur_pull['time']

  # Calculating time-decaying Relationship
  PRvector = [cal_decayRelation(find_PRlist(OS, rev, cur_time), cur_time) for rev in all_reviewers]

  # Calculating time-decaying File Similarity
  PRvector.extend([cal_decayFileSimilarity(find_PRlist(prev_pulls, rev, cur_time), cur_time, cur_pull) for rev in all_reviewers])

  if t % 200 == 0:
    print("PR Vector #", t, ": ", PRvector)

  # Save to dataframe
  vectors.append(PRvector)
  labels.append(cur_pull['reviewersId'][0])


print("PR vector size:", len(all_reviewers), "is equal to: 2 *", len(all_reviewers), "=", 2*len(all_reviewers))
print(len(vectors), len(labels))

PR Vector # 0 :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
PR Vector # 200 :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Create a dataframe of vectors and labels(reviewerIDs)
df = pd.DataFrame()
df['PRvector'] = vectors
df['reviewerID'] = labels
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1060119
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1060119
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1148713
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",458642
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1060119


In [ ]:
# Save dataframe as csv file
df.to_csv('dataframe.csv')

In [ ]:
# Save dataframe into Drive
src = "./dataframe.csv"
dst = "/content/drive/MyDrive/Colab Notebooks/My Project/"+dataset+"_FeatureVectors.csv"
shutil.move(src, dst)

'/content/drive/MyDrive/Colab Notebooks/My Project/brackets_FeatureVectors.csv'

### Experiment & Result

In [ ]:
def pred_classes(index):
  all_reviewers = set()   
  for pr in final_data[0:index]:
    for r in pr['reviewersId']:
      all_reviewers.add(int(r))

  return sorted(all_reviewers)

In [ ]:
# Function to write predicted top-k reviewers for each test into a JSON file
def save_predictions(pred_list):
  with open('output.json', 'a') as outfile:
    json.dump(pred_list, outfile, indent=4)

In [ ]:
def create_pred_obj(group_num, test_num, pred_revs, actual_rev):
  pred_obj = {}
  pred_obj['changeId'] = str(group_num) + "g" + str(test_num)       
  pred_obj['actual'] = [actual_rev]
  pred_obj['recommended'] = pred_revs
  return pred_obj

### Main

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project/'+dataset+'_FeatureVectors.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)
df.head()

,PRvector,reviewerID
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1060119
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1060119
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1148713
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",458642
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1060119


In [ ]:
# PRs created in the same month should be in the same group
groups = []

start_year = final_data[0]['time'].year
start_month = final_data[0]['time'].month
group = []

for i in range(0, len(final_data)):
  if (final_data[i]['time'].year == start_year) and (final_data[i]['time'].month == start_month):
    group.append(df.loc[i, :])
    if(i == len(final_data)-1): # for the last PR
      groups.append(group)
  else:
    groups.append(group)
    group = []
    start_year = final_data[i]['time'].year
    start_month = final_data[i]['time'].month
    group.append(df.loc[i, :])

len(groups)

56

In [ ]:
rounds_num = len(groups)-1

RFmodel = RandomForestClassifier(n_estimators=12, max_depth=None, random_state=7)
NBmodel = GaussianNB()

train = groups[0]
# setting X, y to use for model construction
X = [ast.literal_eval(train[j]["PRvector"]) for j in range(0, len(train))]
y = [train[j]["reviewerID"] for j in range(0, len(train))]

for i in range(0, rounds_num):

  predictions_list = []  # final predicted reviewers for each test PR

  print("********** Test Group number", i+1, "**********")

  test = groups[i+1]
  X_test = [ast.literal_eval(test[j]["PRvector"]) for j in range(0, len(test))]
  y_test = [test[j]["reviewerID"] for j in range(0, len(test))]

  # use classifiers to find probabilities for each test
  for t in range(0, len(X_test)):

    # Random Forest classifier
    RFmodel.fit(X, y)
    RF_prob = RFmodel.predict_proba(np.reshape(X_test[t], (1, -1)))

    # Naive Bayes classifier
    NBmodel.fit(X, y)
    NB_prob = NBmodel.predict_proba(np.reshape(X_test[t], (1, -1)))
    
    # Composer
    comp_prob = (RF_prob * gama) + (NB_prob * (1-gama))
    comp_prob = [round(c, 2) for c in comp_prob[0]]

    # dictionary of each label and its probability
    results = dict(zip(pred_classes(len(X)), comp_prob))  
    # sort predicted reviewers by probability values
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)  
    # extract maximum 10 first predictions
    pred_revs = [str(tup[0]) for tup in sorted_results[0:10] if (tup[1]!=0)]

    if t % 10 == 0:
      print("Test number", t)
      print("Predicted reviewers:", pred_revs)
      print("Actual reviewer:", str(y_test[t]))
      print("-----------------")

    # make object of real and predicted reviewers
    predictions_list.append(create_pred_obj(i+1, t, pred_revs, str(y_test[t])))

    # append the test PR to trainset
    X.append(X_test[t])
    y.append(y_test[t])


  # save objects into a file
  save_predictions(predictions_list)


********** Test Group number 1 **********
Test number 0
Predicted reviewers: ['458642', '1060119', '1148713']
Actual reviewer: 458642
-----------------
Test number 10
Predicted reviewers: ['1148713', '1060119', '458642']
Actual reviewer: 908679
-----------------
Test number 20
Predicted reviewers: ['1060119', '1148713', '1197510', '458642']
Actual reviewer: 1197510
-----------------
Test number 30
Predicted reviewers: ['1060119', '458642']
Actual reviewer: 1060119
-----------------
Test number 40
Predicted reviewers: ['1060119', '458642', '1197510']
Actual reviewer: 697759
-----------------
********** Test Group number 2 **********
Test number 0
Predicted reviewers: ['1060119', '1197510', '458642', '1184639']
Actual reviewer: 1197510
-----------------
Test number 10
Predicted reviewers: ['1060119', '697759', '1148713']
Actual reviewer: 1060119
-----------------
Test number 20
Predicted reviewers: ['1060119', '458642', '1184639', '1148713', '1197510']
Actual reviewer: 1148713
----------